In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB, MultinomialNB
import plotly.graph_objects as go

# Leitura de dados

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
frames = [df_train,df_test]
df = pd.concat(frames)

In [ ]:
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129880 entries, 0 to 25975
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         129880 non-null  int64  
 1   id                                 129880 non-null  int64  
 2   Gender                             129880 non-null  object 
 3   Customer Type                      129880 non-null  object 
 4   Age                                129880 non-null  int64  
 5   Type of Travel                     129880 non-null  object 
 6   Class                              129880 non-null  object 
 7   Flight Distance                    129880 non-null  int64  
 8   Inflight wifi service              129880 non-null  int64  
 9   Departure/Arrival time convenient  129880 non-null  int64  
 10  Ease of Online booking             129880 non-null  int64  
 11  Gate location                      12988

In [ ]:
df.isna().sum()

Unnamed: 0                             0
id                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             393
satisfaction    

In [ ]:
pd.DataFrame()

""


In [ ]:
px.bar(pd.DataFrame(df['satisfaction'].value_counts()))

# Pré Processamento

## Exclusão de colunas não necessarias, Padronização e Redução de dimensionalidade

In [ ]:
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].mean())

In [ ]:
df_original= df.copy()

In [ ]:
#df['Delay'] = df['Arrival Delay in Minutes'] + df['Departure Delay in Minutes']

In [ ]:
#df.drop(['Unnamed: 0', 'id', 'Gender', 'Age', 'Arrival Delay in Minutes', 'Departure Delay in Minutes'], axis=1, inplace=True)

In [ ]:
df.drop(['Unnamed: 0', 'id', 'Gender'], axis=1, inplace=True)

In [ ]:
le = LabelEncoder()
df['satisfaction'] = le.fit_transform(df['satisfaction'])
df['Customer Type'] = 1 - le.fit_transform(df['Customer Type']) # o 1- foi utilizado para podeer atribuir o valor 0 para os clientes não leais e 1 para os leais, caso não tivesse isto estava sendo feito ao contrario

In [ ]:
le = LabelEncoder()
df_original['satisfaction'] = le.fit_transform(df_original['satisfaction'])
df_original['Customer Type'] = 1 - le.fit_transform(df_original['Customer Type']) # o 1- foi utilizado para podeer atribuir o valor 0 para os clientes não leais e 1 para os leais, caso não tivesse isto estava sendo feito ao contrario

In [ ]:
oh = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore'),
    ['Class','Type of Travel']),
    remainder='passthrough',
    verbose_feature_names_out= False)
df = oh.fit_transform(df)
df = pd.DataFrame(df, columns=oh.get_feature_names_out())


In [ ]:
oh = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore'),
    ['Class','Type of Travel', 'Gender']),
    remainder='passthrough',
    verbose_feature_names_out= False)
df_original = oh.fit_transform(df_original)
df_original = pd.DataFrame(df_original, columns=oh.get_feature_names_out())


In [ ]:
df.head()

,Class_Business,Class_Eco,Class_Eco Plus,Type of Travel_Business travel,Type of Travel_Personal Travel,Customer Type,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0.0,0.0,1.0,0.0,1.0,1.0,13.0,460.0,3.0,4.0,...,5.0,4.0,3.0,4.0,4.0,5.0,5.0,25.0,18.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,25.0,235.0,3.0,2.0,...,1.0,1.0,5.0,3.0,1.0,4.0,1.0,1.0,6.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,26.0,1142.0,2.0,2.0,...,5.0,4.0,3.0,4.0,4.0,4.0,5.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,1.0,25.0,562.0,2.0,5.0,...,2.0,2.0,5.0,3.0,1.0,4.0,2.0,11.0,9.0,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,61.0,214.0,3.0,3.0,...,3.0,3.0,4.0,4.0,3.0,3.0,3.0,0.0,0.0,1.0


In [ ]:
fig = px.imshow(df.corr(), text_auto=True,aspect="auto")
fig.show()

In [ ]:
df.corr()

,Class_Business,Class_Eco,Class_Eco Plus,Type of Travel_Business travel,Type of Travel_Personal Travel,Customer Type,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
Class_Business,1.000000,-0.864761,-0.267779,0.552173,-0.552173,0.085685,0.138731,0.466594,0.034033,-0.093931,...,0.202002,0.222942,0.214844,0.172724,0.162488,0.166163,0.142704,-0.010296,-0.014909,0.502476
Class_Eco,-0.864761,1.000000,-0.252278,-0.501083,0.501083,-0.118400,-0.132597,-0.403881,-0.037176,0.079799,...,-0.176933,-0.184657,-0.183162,-0.138829,-0.129629,-0.134774,-0.124709,0.008775,0.012282,-0.449386
Class_Eco Plus,-0.267779,-0.252278,1.000000,-0.102611,0.102611,0.062055,-0.012922,-0.124198,0.005747,0.027894,...,-0.049776,-0.075307,-0.062571,-0.066465,-0.064393,-0.061603,-0.035711,0.003004,0.005165,-0.106034
Type of Travel_Business travel,0.552173,-0.501083,-0.102611,1.000000,-1.000000,-0.308236,0.044808,0.266792,0.105865,-0.257102,...,0.152936,0.059794,0.139612,0.033012,-0.016247,0.023538,0.084615,0.005913,0.005820,0.449861
Type of Travel_Personal Travel,-0.552173,0.501083,0.102611,-1.000000,1.000000,0.308236,-0.044808,-0.266792,-0.105865,0.257102,...,-0.152936,-0.059794,-0.139612,-0.033012,0.016247,-0.023538,-0.084615,-0.005913,-0.005820,-0.449861
Customer Type,0.085685,-0.118400,0.062055,-0.308236,0.308236,1.000000,0.284172,0.226021,0.005757,0.206873,...,0.106001,0.054172,0.046841,-0.024874,0.031243,-0.023292,0.081302,-0.003859,-0.004724,0.186017
Age,0.138731,-0.132597,-0.012922,0.044808,-0.044808,0.284172,1.000000,0.099459,0.016116,0.036960,...,0.074947,0.057078,0.039119,-0.047991,0.033475,-0.051347,0.052565,-0.009041,-0.011229,0.134091
Flight Distance,0.466594,-0.403881,-0.124198,0.266792,-0.266792,0.226021,0.099459,1.000000,0.006701,-0.018914,...,0.130507,0.111194,0.134533,0.064855,0.073608,0.059316,0.095648,0.002402,-0.001932,0.298085
Inflight wifi service,0.034033,-0.037176,0.005747,0.105865,-0.105865,0.005757,0.016116,0.006701,1.000000,0.344915,...,0.207802,0.119928,0.160317,0.120376,0.043762,0.110029,0.131300,-0.015946,-0.017721,0.283460
Departure/Arrival time convenient,-0.093931,0.079799,0.027894,-0.257102,0.257102,0.206873,0.036960,-0.018914,0.344915,1.000000,...,-0.008380,0.067297,0.010617,0.070833,0.091132,0.072195,0.009862,0.000778,-0.000941,-0.054270


In [ ]:
df.drop(['Departure/Arrival time convenient', 'Gate location', 'Departure Delay in Minutes', 'Arrival Delay in Minutes'], axis=1, inplace=True) #removendo atributos que não tem correlação com nosso alvo, de acordo com a tabela

## Padronização dos dados

In [ ]:
df.describe()

,Class_Business,Class_Eco,Class_Eco Plus,Type of Travel_Business travel,Type of Travel_Personal Travel,Customer Type,Age,Flight Distance,Inflight wifi service,Ease of Online booking,...,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction
count,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,...,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000
mean,0.478596,0.448945,0.072459,0.690584,0.309416,0.816908,39.427957,1190.316392,2.728696,2.756876,...,3.252633,3.441361,3.358077,3.383023,3.350878,3.632114,3.306267,3.642193,3.286326,0.434463
std,0.499544,0.497388,0.259248,0.462255,0.462255,0.386743,15.119360,997.452477,1.329340,1.401740,...,1.350719,1.319289,1.334049,1.287099,1.316252,1.180025,1.266185,1.176669,1.313682,0.495688
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,31.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,27.000000,414.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,40.000000,844.000000,3.000000,3.000000,...,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000
75%,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,51.000000,1744.000000,4.000000,4.000000,...,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,85.000000,4983.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000


In [ ]:
df[['Flight Distance', 'Inflight wifi service', 'Ease of Online booking', 'Food and drink', 'Online boarding', 'Seat comfort',
    'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'Age']].describe()

,Flight Distance,Inflight wifi service,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Age
count,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000
mean,1190.316392,2.728696,2.756876,3.204774,3.252633,3.441361,3.358077,3.383023,3.350878,3.632114,3.306267,3.642193,3.286326,39.427957
std,997.452477,1.329340,1.401740,1.329933,1.350719,1.319289,1.334049,1.287099,1.316252,1.180025,1.266185,1.176669,1.313682,15.119360
min,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,7.000000
25%,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,27.000000
50%,844.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,40.000000
75%,1744.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,51.000000
max,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,85.000000


In [ ]:
scaler = make_column_transformer(
    (StandardScaler(),
    ['Flight Distance', 'Inflight wifi service', 'Ease of Online booking', 'Food and drink', 'Online boarding', 'Seat comfort',
    'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness','Age']),
    remainder='passthrough',
    verbose_feature_names_out= False)
df = scaler.fit_transform(df)
df = pd.DataFrame(df, columns=scaler.get_feature_names_out())


In [ ]:
df[['Flight Distance', 'Inflight wifi service', 'Ease of Online booking', 'Food and drink', 'Online boarding', 'Seat comfort',
    'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'Age']].describe()

,Flight Distance,Inflight wifi service,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Age
count,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05,1.298800e+05
mean,9.168999e-17,-1.373162e-17,-1.524155e-16,7.998256e-17,6.061606e-17,-2.308662e-17,-1.750644e-17,1.089229e-16,6.663390e-17,-7.493578e-17,-7.899782e-17,1.165273e-16,3.539584e-17,-9.179941e-17
std,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00
min,-1.162282e+00,-2.052677e+00,-1.966761e+00,-2.409735e+00,-2.408085e+00,-2.608507e+00,-2.517216e+00,-2.628419e+00,-2.545782e+00,-2.230567e+00,-2.611213e+00,-3.095354e+00,-2.501624e+00,-2.144805e+00
25%,-7.783021e-01,-5.481656e-01,-5.399566e-01,-9.058939e-01,-9.273863e-01,-1.092533e+00,-1.018015e+00,-1.074531e+00,-1.026310e+00,-5.356810e-01,-2.418828e-01,-5.457739e-01,-9.791796e-01,-8.219928e-01
50%,-3.472022e-01,2.040902e-01,1.734455e-01,-1.539735e-01,-1.870368e-01,4.234409e-01,4.811861e-01,4.793566e-01,4.931616e-01,3.117623e-01,-2.418828e-01,3.040860e-01,-2.179575e-01,3.783531e-02
75%,5.550999e-01,9.563460e-01,8.868475e-01,5.979470e-01,5.533126e-01,1.181428e+00,4.811861e-01,4.793566e-01,4.931616e-01,1.159206e+00,5.478940e-01,1.153946e+00,5.432645e-01,7.653821e-01
max,3.802385e+00,1.708602e+00,1.600249e+00,1.349867e+00,1.293662e+00,1.181428e+00,1.230787e+00,1.256300e+00,1.252898e+00,1.159206e+00,1.337671e+00,1.153946e+00,1.304487e+00,3.014163e+00


In [ ]:
df.head()

,Flight Distance,Inflight wifi service,Ease of Online booking,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,...,Inflight service,Cleanliness,Age,Class_Business,Class_Eco,Class_Eco Plus,Type of Travel_Business travel,Type of Travel_Personal Travel,Customer Type,satisfaction
0,-0.732184,0.204090,0.173445,1.349867,-0.187037,1.181428,1.230787,0.479357,-0.266574,0.311762,...,1.153946,1.304487,-1.747961,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,-0.957760,0.204090,0.173445,-1.657814,-0.187037,-1.850520,-1.767616,-1.851475,1.252898,-0.535681,...,0.304086,-1.740402,-0.954274,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.048440,-0.548166,-0.539957,1.349867,1.293662,1.181428,1.230787,0.479357,-0.266574,0.311762,...,0.304086,1.304487,-0.888133,1.0,0.0,0.0,1.0,0.0,1.0,1.0
3,-0.629924,-0.548166,1.600249,-0.905894,-0.927386,-1.092533,-1.018015,-1.074531,1.252898,-0.535681,...,0.304086,-0.979180,-0.954274,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,-0.978814,0.204090,0.173445,0.597947,1.293662,1.181428,-0.268414,-0.297587,0.493162,0.311762,...,-0.545774,-0.217958,1.426788,1.0,0.0,0.0,1.0,0.0,1.0,1.0


## Classificação

In [ ]:
def scores(model, X, y):
    kf = KFold(10,shuffle=False) #shuffle false pois queremos sempre repetir os mesmos valores para todo algoritmo utilizado, para a comparação ser justa
    predicted = np.empty([0], dtype=int)
    actual = np.empty([0], dtype=int)
    for _, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
        model = model.fit(X_train, y_train)
        predict = model.predict(X_test)

        predicted = np.append(predicted, predict)
        actual = np.append(actual, y_test)

    return actual, predicted

In [ ]:
def plot_confusion_matrix(actual_classes, predicted_classes):

    matrix = confusion_matrix(actual_classes, predicted_classes)

    plt.figure(figsize=(12.8,6))
    fig = px.imshow(matrix, text_auto=True,aspect="auto", labels=dict(x='Predict', y='Actual'), x=['Satisfeito', 'Não Satisfeito'], y=['Satisfeito', 'Não Satisfeito'], color_continuous_scale='blues')
    fig.show()

    AC1 = (matrix[0,0]+matrix[1,1])/matrix.sum()
    Prec1 = matrix.diagonal()/matrix.sum(axis=0)
    Rev1 = matrix.diagonal()/matrix.sum(axis=1)
    MF1 = 2*(Prec1*Rev1)/(Prec1+Rev1)

    print('Acurácia: {:.2f}'.format(AC1))
    print('Precisão: {:.2f}'.format(Prec1.mean()))
    print('Revocação: {:.2f}'.format(Rev1.mean()))
    print('Medida-F: {:.2f}'.format(MF1.mean()))


In [ ]:
X = df.drop("satisfaction", axis=1)
y = df["satisfaction"]
X = X.values
y = y.values

In [ ]:
X_original = df_original.drop("satisfaction", axis=1)
y_original = df_original["satisfaction"]
X_original = X_original.values
y_original = y_original.values

###**Árvore de Descisão**#


#### Com pré-processamento

In [ ]:
tree = DecisionTreeClassifier(criterion='entropy', random_state=42)
actual, predicted  = scores(tree, X, y)
plot_confusion_matrix(actual, predicted)

Acurácia: 0.95
Precisão: 0.95
Revocação: 0.95
Medida-F: 0.95


<Figure size 1280x600 with 0 Axes>

#### Sem  pré processamento

apenas com remoção dos nulos e transformando os valores de string em int, se não o programa não roda

In [ ]:
tree = DecisionTreeClassifier(criterion='entropy', random_state=42)
actual, predicted  = scores(tree, X_original, y_original)
plot_confusion_matrix(actual, predicted)

Acurácia: 0.95
Precisão: 0.95
Revocação: 0.95
Medida-F: 0.95


<Figure size 1280x600 with 0 Axes>

### KNN



#### Dataset pré processado

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
actual, predicted  = scores(knn, X, y)
plot_confusion_matrix(actual, predicted)

Acurácia: 0.93
Precisão: 0.94
Revocação: 0.93
Medida-F: 0.93


<Figure size 1280x600 with 0 Axes>

#### Sem  pré processamento

apenas com remoção dos nulos e transformando os valores de string em int, se não o programa não roda

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
actual, predicted  = scores(knn, X_original, y_original)
plot_confusion_matrix(actual, predicted)

Acurácia: 0.57
Precisão: 0.56
Revocação: 0.56
Medida-F: 0.56


<Figure size 1280x600 with 0 Axes>

#**Classificação - Bayes - Gaussian Naive Bayes**#

Método de classificação de Bayes Ingênuo Gaussiano


#### Dataset pré processado

In [ ]:
gnb = GaussianNB()
actual, predicted  = scores(gnb, X, y)
plot_confusion_matrix(actual, predicted)

Acurácia: 0.85
Precisão: 0.84
Revocação: 0.84
Medida-F: 0.84


<Figure size 1280x600 with 0 Axes>

#### Sem  pré processamento

apenas com remoção dos nulos e transformando os valores de string em int, se não o programa não roda

In [ ]:
gnb2 = GaussianNB()
actual, predicted  = scores(gnb2, X_original, y_original)
plot_confusion_matrix(actual, predicted)

Acurácia: 0.81
Precisão: 0.81
Revocação: 0.81
Medida-F: 0.81


<Figure size 1280x600 with 0 Axes>

#### Bayes Multinomial Sem Pré Processamento

Com atributos com muitas entradas como idade, distancia e atraso

In [ ]:
mnb = MultinomialNB()
actual, predicted  = scores(mnb, X_original, y_original)
plot_confusion_matrix(actual, predicted)

Acurácia: 0.56
Precisão: 0.56
Revocação: 0.56
Medida-F: 0.56


<Figure size 1280x600 with 0 Axes>

Retirando estes atributos como ID, Distancia de Voo, Idade e Atrasos, o algoritmo multinomial tem desempenho diferente:

In [ ]:

df_novo = df_original.copy()
df_novo.drop(['Unnamed: 0', 'id', 'Flight Distance', 'Age', 'Departure Delay in Minutes', 'Arrival Delay in Minutes'], axis=1, inplace=True)
df_novo.head()

X_novo = df_novo.drop("satisfaction", axis=1)
y_novo = df_novo["satisfaction"]
X_novo = X_novo.values
y_novo = y_novo.values

mnb2 = MultinomialNB()
actual, predicted  = scores(mnb2, X_novo, y_novo)
plot_confusion_matrix(actual, predicted)

Acurácia: 0.79
Precisão: 0.79
Revocação: 0.79
Medida-F: 0.79


<Figure size 1280x600 with 0 Axes>